In [1]:
import sys
import os
sys.path.append('../..')
import torch
import torchvision
from src.models import ResNet18
from src.transforms import ImageResizer,make_patches
import matplotlib.pyplot as plt
import pandas as pd
from torchsummary import summary

/home/abdelnour/anaconda3/envs/ml/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/abdelnour/anaconda3/envs/ml/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
model = ResNet18(n_classes=3) # the weights should be loaded from a file

In [ ]:
dataset = torchvision.datasets.ImageFolder(
    root="<patched-images-path>"
)

In [ ]:
dataloader = torch.utils.data.DataLoader(dataset=dataset,batch_size=16,shuffle=True)

In [ ]:
def predict(
    dataloader : torch.utils.data.DataLoader, 
    model : torch.nn.Module
) -> pd.DataFrame :
    
    result = {}
    
    for x, y in dataloader:
        
        y_hat = model(x)
        y_hat = torch.nn.functional.softmax(x, dim=1)

